## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [51]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df1 = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df1.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [4]:
df1 = df1.loc[:, ['year', 'title', 'domestic_gross', 'foreign_gross']]
df1 = df1.sort_values('year', ascending = False)
df1

,year,title,domestic_gross,foreign_gross
3386,2018,An Actor Prepares,1700.0,NaN
3183,2018,On the Basis of Sex,24600000.0,13600000
3176,2018,Tyler Perry's Acrimony,43500000.0,2900000
3177,2018,Mary Queen of Scots,16500000.0,29900000
3178,2018,The Possession of Hannah Grace,14800000.0,28200000
...,...,...,...,...
220,2010,After.Life,109000.0,1900000
221,2010,Cairo Time,1600000.0,391000
222,2010,Flipped,1800000.0,NaN
223,2010,Guzaarish,1000000.0,695000


In [5]:
df1.duplicated().value_counts()

False    3387
dtype: int64

In [6]:
df2 = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter = '\t')
df2.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [8]:
df2 = df2.loc[:, ['theater_date', 'genre', 'director']]
df2

,theater_date,genre,director
0,"Oct 9, 1971",Action and Adventure|Classics|Drama,William Friedkin
1,"Aug 17, 2012",Drama|Science Fiction and Fantasy,David Cronenberg
2,"Sep 13, 1996",Drama|Musical and Performing Arts,Allison Anders
3,"Dec 9, 1994",Drama|Mystery and Suspense,Barry Levinson
4,NaN,Drama|Romance,Rodney Bennett
...,...,...,...
1555,"Aug 18, 2006",Action and Adventure|Horror|Mystery and Suspense,NaN
1556,"Jul 23, 1993",Comedy|Science Fiction and Fantasy,Steve Barron
1557,"Jan 1, 1962",Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas
1558,"Apr 1, 1993",Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans


In [9]:
df2.duplicated().value_counts()

False    1518
True       42
dtype: int64

In [10]:
df2.dropna(inplace = True)
df2['theater_date'] = df2.theater_date.map(lambda x: x[7:])
df2 = df2.sort_values('theater_date', ascending = False)
df2

,theater_date,genre,director
1064,2018,Drama,Desiree Akhavan
1201,2018,Action and Adventure|Comedy|Kids and Family,Marc Forster
686,2018,Drama,Sean McNamara
1302,2018,Drama,Clint Eastwood
319,2018,Mystery and Suspense|Science Fiction and Fantasy,Jennifer Yuh Nelson
...,...,...,...
1129,1934,Classics|Drama|Science Fiction and Fantasy,Fritz Lang
1183,1933,Action and Adventure|Classics|Western,Henry Hathaway
183,1932,Classics|Comedy|Musical and Performing Arts|Ro...,George Cukor|Ernst Lubitsch
656,1932,Classics|Drama|Mystery and Suspense,Mervyn Le Roy


In [52]:
df3 = pd.read_csv('zippedData/tmdb.movies.csv.gz')
df3.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [53]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [54]:
df3 = df3.loc[:, ['release_date', 'title', 'vote_average']]
df3['release_date'] = df3.release_date.map(lambda x: x[:4])
df3 = df3.sort_values('release_date', ascending = False)
df3

,release_date,title,vote_average
26057,2020,Murdery Christmas,0.0
24384,2019,Piercing,5.9
25429,2019,Bilby,5.0
24933,2019,Late Afternoon,7.7
24764,2019,Holiday,5.5
...,...,...,...
11192,1946,The Best Years of Our Lives,7.8
26345,1939,How Walt Disney Cartoons Are Made,7.3
3580,1936,Le Bonheur,8.7
21758,1933,The Vampire Bat,5.6


In [55]:
df3.duplicated().value_counts()

False    25492
True      1025
dtype: int64

In [56]:
df3['release_date'] = df3['release_date'].astype(str).astype(int)
df3 = df3[(df3['release_date'] >= 2010)]
df3

,release_date,title,vote_average
26057,2020,Murdery Christmas,0.0
24384,2019,Piercing,5.9
25429,2019,Bilby,5.0
24933,2019,Late Afternoon,7.7
24764,2019,Holiday,5.5
...,...,...,...
1675,2010,No Fun City,4.3
1729,2010,Mountains That Take Wing: Angela Davis & Yuri ...,6.0
1731,2010,Unrequited Love,6.0
1640,2010,Deuce of Spades,5.3


In [57]:
df3_renamed = df3.rename(columns = {'title' : 'movie', 'release_date' : 'year', 'vote_average' : 'rating'})
df3_renamed

,year,movie,rating
26057,2020,Murdery Christmas,0.0
24384,2019,Piercing,5.9
25429,2019,Bilby,5.0
24933,2019,Late Afternoon,7.7
24764,2019,Holiday,5.5
...,...,...,...
1675,2010,No Fun City,4.3
1729,2010,Mountains That Take Wing: Angela Davis & Yuri ...,6.0
1731,2010,Unrequited Love,6.0
1640,2010,Deuce of Spades,5.3


In [58]:
df4 = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
df4.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [59]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [60]:
df4 = df4.loc[:, ['release_date', 'movie', 'production_budget', 'worldwide_gross']]
df4['release_date'] = df4.release_date.map(lambda x: x[7:])
df4 = df4.sort_values('release_date', ascending = False)
df4

,release_date,movie,production_budget,worldwide_gross
3633,2019,The Best of Enemies,"$10,000,000","$10,205,616"
3915,2019,El Chicano,"$8,000,000","$700,261"
580,2019,The Secret Life of Pets 2,"$80,000,000","$113,351,496"
496,2019,Shazam!,"$85,000,000","$362,899,733"
95,2019,Captain Marvel,"$175,000,000","$1,123,061,550"
...,...,...,...,...
4984,1927,Wings,"$2,000,000",$0
5606,1925,The Big Parade,"$245,000","$22,000,000"
4569,1925,Ben-Hur: A Tale of the Christ,"$3,900,000","$9,000,000"
5683,1920,Over the Hill to the Poorhouse,"$100,000","$3,000,000"


In [61]:
df4.duplicated().value_counts()

False    5782
dtype: int64

In [62]:
df4['release_date'] = df4['release_date'].astype(str).astype(int)
df4 = df4[df4['release_date'] >= 2010]
df4

,release_date,movie,production_budget,worldwide_gross
3633,2019,The Best of Enemies,"$10,000,000","$10,205,616"
3915,2019,El Chicano,"$8,000,000","$700,261"
580,2019,The Secret Life of Pets 2,"$80,000,000","$113,351,496"
496,2019,Shazam!,"$85,000,000","$362,899,733"
95,2019,Captain Marvel,"$175,000,000","$1,123,061,550"
...,...,...,...,...
4572,2010,The Good Heart,"$3,800,000","$340,930"
2352,2010,Faster,"$24,000,000","$35,792,945"
5309,2010,Zombies of Mass Destruction,"$1,000,000",$0
3711,2010,Welcome to the Rileys,"$10,000,000","$355,919"


In [63]:
df4_renamed = df4.rename(columns = {'release_date' : 'year'})
df4_renamed

,year,movie,production_budget,worldwide_gross
3633,2019,The Best of Enemies,"$10,000,000","$10,205,616"
3915,2019,El Chicano,"$8,000,000","$700,261"
580,2019,The Secret Life of Pets 2,"$80,000,000","$113,351,496"
496,2019,Shazam!,"$85,000,000","$362,899,733"
95,2019,Captain Marvel,"$175,000,000","$1,123,061,550"
...,...,...,...,...
4572,2010,The Good Heart,"$3,800,000","$340,930"
2352,2010,Faster,"$24,000,000","$35,792,945"
5309,2010,Zombies of Mass Destruction,"$1,000,000",$0
3711,2010,Welcome to the Rileys,"$10,000,000","$355,919"


In [64]:
import sqlite3
conn = sqlite3.connect('zippedData/im.db')
cur = conn.cursor()
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
table_names = cur.fetchall()
table_names

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [65]:
p = """
SELECT mb.primary_title AS movie, 
       mb.genres AS genre, 
       mb.start_year AS year, 
       mr.averagerating AS rating, 
       p.primary_name AS director
FROM movie_basics AS mb
JOIN movie_ratings AS mr
    USING(movie_id)
JOIN directors as d
    USING(movie_id)
JOIN persons AS p
    USING(person_id)
ORDER BY start_year DESC;
"""
df5 = pd.read_sql(p, conn)
df5

,movie,genre,year,rating,director
0,One Day Before the Rainy Season,"Biography,Drama",2019,7.2,Mani Kaul
1,On kadin,Drama,2019,7.1,Serif Gören
2,Abolição,Documentary,2019,7.4,Zózimo Bulbul
3,Alita: Battle Angel,"Action,Adventure,Sci-Fi",2019,7.5,Robert Rodriguez
4,Alita: Battle Angel,"Action,Adventure,Sci-Fi",2019,7.5,Robert Rodriguez
...,...,...,...,...,...
181382,Hatchet Sisters,Horror,2010,5.4,Joe Black
181383,Hatchet Sisters,Horror,2010,5.4,Joe Black
181384,Hatchet Sisters,Horror,2010,5.4,Joe Black
181385,Peculiar Pets,Documentary,2010,6.4,Adrian Cale


In [66]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181387 entries, 0 to 181386
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   movie     181387 non-null  object 
 1   genre     180047 non-null  object 
 2   year      181387 non-null  int64  
 3   rating    181387 non-null  float64
 4   director  181387 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 6.9+ MB


In [67]:
duplicates = df5[df5.duplicated()]
duplicates

,movie,genre,year,rating,director
4,Alita: Battle Angel,"Action,Adventure,Sci-Fi",2019,7.5,Robert Rodriguez
5,Alita: Battle Angel,"Action,Adventure,Sci-Fi",2019,7.5,Robert Rodriguez
8,Shazam!,"Action,Adventure,Comedy",2019,7.4,David F. Sandberg
9,Shazam!,"Action,Adventure,Comedy",2019,7.4,David F. Sandberg
10,Shazam!,"Action,Adventure,Comedy",2019,7.4,David F. Sandberg
...,...,...,...,...,...
181372,Mago Digo Dai,Comedy,2010,7.2,Sri Lal Priyadeva
181380,Hatchet Sisters,Horror,2010,5.4,Jeremy Tidwell
181381,Hatchet Sisters,Horror,2010,5.4,Jeremy Tidwell
181383,Hatchet Sisters,Horror,2010,5.4,Joe Black


In [68]:
df5 = df5.drop_duplicates()
df5

,movie,genre,year,rating,director
0,One Day Before the Rainy Season,"Biography,Drama",2019,7.2,Mani Kaul
1,On kadin,Drama,2019,7.1,Serif Gören
2,Abolição,Documentary,2019,7.4,Zózimo Bulbul
3,Alita: Battle Angel,"Action,Adventure,Sci-Fi",2019,7.5,Robert Rodriguez
6,Surviving in L.A.,"Comedy,Drama,Romance",2019,8.7,Abbi Lake O'Neill
...,...,...,...,...,...
181378,Setteekh,"Horror,Mystery,Thriller",2010,5.1,Evgeniy Pavlov
181379,Hatchet Sisters,Horror,2010,5.4,Jeremy Tidwell
181382,Hatchet Sisters,Horror,2010,5.4,Joe Black
181385,Peculiar Pets,Documentary,2010,6.4,Adrian Cale


In [69]:
df5.isna().sum()

movie         0
genre       798
year          0
rating        0
director      0
dtype: int64

In [71]:
df5_filled = df5.fillna({'genre': 'Unknown'})
df5_filled 

,movie,genre,year,rating,director
0,One Day Before the Rainy Season,"Biography,Drama",2019,7.2,Mani Kaul
1,On kadin,Drama,2019,7.1,Serif Gören
2,Abolição,Documentary,2019,7.4,Zózimo Bulbul
3,Alita: Battle Angel,"Action,Adventure,Sci-Fi",2019,7.5,Robert Rodriguez
6,Surviving in L.A.,"Comedy,Drama,Romance",2019,8.7,Abbi Lake O'Neill
...,...,...,...,...,...
181378,Setteekh,"Horror,Mystery,Thriller",2010,5.1,Evgeniy Pavlov
181379,Hatchet Sisters,Horror,2010,5.4,Jeremy Tidwell
181382,Hatchet Sisters,Horror,2010,5.4,Joe Black
181385,Peculiar Pets,Documentary,2010,6.4,Adrian Cale


In [74]:
df5['genre'] = df5_filled['genre'].tolist()
df5['genre'] = df5_filled['genre'].map(lambda x: x.split(','))
df5.explode('genre')

,movie,genre,year,rating,director
0,One Day Before the Rainy Season,Biography,2019,7.2,Mani Kaul
0,One Day Before the Rainy Season,Drama,2019,7.2,Mani Kaul
1,On kadin,Drama,2019,7.1,Serif Gören
2,Abolição,Documentary,2019,7.4,Zózimo Bulbul
3,Alita: Battle Angel,Action,2019,7.5,Robert Rodriguez
...,...,...,...,...,...
181378,Setteekh,Thriller,2010,5.1,Evgeniy Pavlov
181379,Hatchet Sisters,Horror,2010,5.4,Jeremy Tidwell
181382,Hatchet Sisters,Horror,2010,5.4,Joe Black
181385,Peculiar Pets,Documentary,2010,6.4,Adrian Cale


In [75]:
new_df = df5.merge(df3_renamed, on = ['year', 'movie', 'rating']).merge(df4_renamed, on = ['year', 'movie']).explode('genre')
new_df

,movie,genre,year,rating,director,production_budget,worldwide_gross
0,Tomb Raider,Action,2018,6.3,Roar Uthaug,"$90,000,000","$273,477,501"
0,Tomb Raider,Adventure,2018,6.3,Roar Uthaug,"$90,000,000","$273,477,501"
0,Tomb Raider,Fantasy,2018,6.3,Roar Uthaug,"$90,000,000","$273,477,501"
1,Hell Fest,Horror,2018,5.5,Gregory Plotkin,"$5,500,000","$18,119,231"
2,The Sisters Brothers,Adventure,2018,7.0,Jacques Audiard,"$38,000,000","$13,143,056"
...,...,...,...,...,...,...,...
161,Country Strong,Music,2010,6.3,Shana Feste,"$15,000,000","$20,601,987"
162,The Christmas Bunny,Drama,2010,6.1,Tom Seidman,"$250,000",$0
162,The Christmas Bunny,Family,2010,6.1,Tom Seidman,"$250,000",$0
163,The Virginity Hit,Comedy,2010,4.6,Andrew Gurland,"$2,000,000","$636,706"


In [79]:
gross_genre = new_df[['genre', 'worldwide_gross']]
gross_genre

,genre,worldwide_gross
0,Action,"$273,477,501"
0,Adventure,"$273,477,501"
0,Fantasy,"$273,477,501"
1,Horror,"$18,119,231"
2,Adventure,"$13,143,056"
...,...,...
161,Music,"$20,601,987"
162,Drama,$0
162,Family,$0
163,Comedy,"$636,706"
